In [ ]:
%config IPCompleter.greedy=True

# Import required libraries

In [ ]:
from __future__ import print_function

import base64
import datetime

from IPython.display import Image
from IPython.core.display import HTML

import GRANTA_MIScriptingToolkit as gdl
from GRANTA_MIScriptingToolkit import GRANTA_Exception
from GRANTA_MIScriptingToolkit import GRANTA_Exceptions
import getpass

import os
import sys

import xml.dom.minidom

# Enter credential to access Granta system

In [ ]:
try:
    granta_url = 'http://wsancalp01.nist.gov/mi_servicelayer'
    login = input("Type login then Enter :")
    password = getpass.getpass(prompt='Password? : ')
    session = gdl.GRANTA_MISession(granta_url, login, password, '')
    print("Done")
except GRANTA_Exception as e:
    print("Unexpected error:", sys.exc_info()[1])


# Load utility functions

In [ ]:
def do_import(existingRecord, importAttributeValues, importRecordMode="Update", releaseRecord=True):
    """
    Wrapper for a data import
    """
    newRecordData = gdl.ImportRecord(importRecordMode=importRecordMode,
                                     existingRecord=existingRecord,
                                     importAttributeValues=importAttributeValues,
                                     releaseRecord=releaseRecord)
        
    request = gdl.SetRecordAttributesRequest(importRecords=[newRecordData])
    response = session.dataImportService.SetRecordAttributes(request)
    return response

def find_record(gdlSession, dbKey, tableName, recordName):
    """ 
    Finds a record by name; returns first ranked item if multiple results
    :returns: :py:mod:`SearchResult <GRANTA_MIScriptingToolkit.SearchResult>`
    """
    req = gdl.RecordNameSearchRequest(caseSensitiveNames=False, recordName=recordName, populateGUIDs=True, searchShortNames=True, searchFullNames=True)
    req.table = gdl.TableReference(DBKey=dbKey, name=tableName)
    resp = gdlSession.searchService.RecordNameSearch(req)
    return resp.searchResults[0]

def get_record_data(session, recordRef, attribs):
    """ 
    Wrapper for a data Export
    Return a dictionary of record attributes (retrieved from an export), ie, name -> AttributeValue
    """
    exp = session.dataExportService
    req = gdl.GetRecordAttributesByRefRequest()
    req.recordReferences = [recordRef]
    req.attributeReferences = attribs
    resp = exp.GetRecordAttributesByRef(req)
    data = resp.recordData[0]
    return {av.attributeName : av for av in data.attributeValues}

def printn(soap_response):
    """
    Pretty print xml document
    """
    xml_response = xml.dom.minidom.parseString(soap_response) 
    pretty_xml_as_string = xml_response.toprettyxml()
    print(pretty_xml_as_string)

def now_time_string():
    """
    Return date and time as a string
    """
    return datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

def read_binary_data(filepath):
    """
    Returns binary data content of specified file
    """
    with open(filepath, "rb") as bin_file:
        return bin_file.read()
print("Function loaded")

# Selection of insert location
 - 'folder' is called 'attribute' in Granta Design's terminology

In [ ]:
dbKey = "MI_NIST_NCAL"
tableName = "Experiments"
folder = "Photos"  # "Other files"
recordName = "E999999-ZZZ-999" 

print("dbKey: {0},\ntableName: {1},\nfolder: {2},\nrecordName: {3}".format(dbKey, tableName, folder, recordName))

# Selection of data

In [ ]:
#filepath = "C:\\Users\\marieros\\OneDrive - National Institute of Standards and Technology (NIST)\\Granta\\notebooks\\test_images\\test_image.jpg"
#comment_string = "this is a comment regarding the file _ created " + now_time_string()

filepath = "C:\\Users\\creuzige\\Downloads\\NIST_logo.png"
comment_string = "Added by Adam " + now_time_string()

display(Image(filename = filepath, width=200, height=200))

print(comment_string)

# Import

- Appends to existing record (see changetype)
- http://support.grantadesign.com/resources/miscriptingtoolkit/python/1.4/index.htm#t=modules%2FTabularDataType.html
- Options for changetype are: InsertBefore, Append, Update, Delete

In [ ]:
target = find_record(session, dbKey, tableName, recordName).recordReference

importTable = gdl.TabularDataType()
newPicture = None
newText = None

contentType = "FILE"
fileName = os.path.split(filepath)[1]

if folder == "Photos":
    importTable.AddColumn("Photos")
    newPicture = gdl.PictureDataType(contentType=contentType)
else:
    importTable.AddColumn("Files")
    newPicture = gdl.FileDataType(contentType=contentType, fileName=fileName)

importTable.AddColumn("Comments")

newPicture.ModifyBinaryData(read_binary_data(filepath))
newText = gdl.LongTextDataType(value=comment_string)


changetype = gdl.TabularDataType.ChangeType.Append
newRow = importTable.CreateUpdateRow(changetype, target.identity)
newRow.cells[0].data = newPicture
newRow.cells[1].data = newText

attributeRef = gdl.AttributeReference(name=folder, partialTableReference=gdl.PartialTableReference(tableName=tableName), DBKey=dbKey)
importAttribute = gdl.ImportAttributeValue(tabularDataValue=importTable, attributeReference=attributeRef)

import_request_response = do_import(existingRecord=target, importAttributeValues=[importAttribute])
importedRecord = import_request_response.recordsImported[0]

print("New record imported: {0}.".format(importedRecord.shortName))

print("\nResponse:")
printn(import_request_response.serviceLayerResponse.responseMessage) 


# Retrieval
 - You will need to edit the numeric location of the data in the HTML call below, based on the response

In [ ]:
importedRef = importedRecord.recordReference
data = get_record_data(session, importedRef, [attributeRef])

dataExportRequest = gdl.GetRecordAttributesByRefRequest(recordReferences = [importedRef], attributeReferences=[attributeRef])
dataExportResponse = session.dataExportService.GetRecordAttributesByRef(dataExportRequest)
soap_response = dataExportResponse.serviceLayerResponse.responseMessage
printn(soap_response)

In [ ]:
HTML('<img src="http://wsancalp01.nist.gov/mi_servicelayer/SingleDatum.svc/databases/MI_NIST_NCAL/data/8566225" />')